#######Primeiro Script , trabalhando com carga de dados e ajuste de registros no Delta Lake
##Carrega os dados do arquivo Json

In [0]:
%python
dfcliente = spark.read.json("/FileStore/tables/cliente/clientes.json");
dfcliente.printSchema()
dfcliente.show()

root
 |-- customer: string (nullable = true)
 |-- date_order: string (nullable = true)
 |-- id: long (nullable = true)
 |-- price: double (nullable = true)
 |-- product: string (nullable = true)
 |-- unit: long (nullable = true)

+--------+----------+---+-----+---------------+----+
|customer|date_order| id|price|        product|unit|
+--------+----------+---+-----+---------------+----+
|  Carlos|2021-01-23|  1|238.0|             TV|   5|
|     Ana|2021-03-22|  2|121.6|AR-CONDICIONADO|   6|
|   Sofia|2021-04-21|  3|415.4|        FREEZER|   7|
|  Sandra|2021-04-23|  4|313.0|             TV|   8|
|  Tereza|2021-05-23|  5|412.0|       FRIGOBAR|   3|
|   Carla|2021-06-25|  6|124.0|           MESA|   1|
|   Sofia|2021-07-25|  7|342.3|        CADEIRA|   1|
+--------+----------+---+-----+---------------+----+



##Cria a tabela temporária com os dos do arquivo json em memória

In [0]:
%python
dfcliente.createOrReplaceTempView("compras_view");
saida =spark.sql("SELECT * FROM compras_view")
saida.show()

+--------+----------+---+-----+---------------+----+
|customer|date_order| id|price|        product|unit|
+--------+----------+---+-----+---------------+----+
|  Carlos|2021-01-23|  1|238.0|             TV|   5|
|     Ana|2021-03-22|  2|121.6|AR-CONDICIONADO|   6|
|   Sofia|2021-04-21|  3|415.4|        FREEZER|   7|
|  Sandra|2021-04-23|  4|313.0|             TV|   8|
|  Tereza|2021-05-23|  5|412.0|       FRIGOBAR|   3|
|   Carla|2021-06-25|  6|124.0|           MESA|   1|
|   Sofia|2021-07-25|  7|342.3|        CADEIRA|   1|
+--------+----------+---+-----+---------------+----+



##Carrega os dados no Delta Lake gerando uma tabela chamada compras, note USING DELTA

In [0]:
%scala
val scrisql = "CREATE OR REPLACE TABLE compras (id STRING, date_order STRING,customer STRING,product STRING,unit INTEGER,price DOUBLE) USING DELTA PARTITIONED BY (date_order) ";
spark.sql(scrisql);

scrisql: String = "CREATE OR REPLACE TABLE compras (id STRING, date_order STRING,customer STRING,product STRING,unit INTEGER,price DOUBLE) USING DELTA PARTITIONED BY (date_order) "
res0: org.apache.spark.sql.DataFrame = []

##Lista os dados do Delta Lake, que estará vazia

In [0]:
%scala
spark.sql("select * from compras").show()

+---+----------+--------+-------+----+-----+
 id|date_order|customer|product|unit|price|
+---+----------+--------+-------+----+-----+
+---+----------+--------+-------+----+-----+

##Criando um merge para carregar os dados da tabela temporário no Delta Lake

In [0]:
%scala
val mergedados = "Merge into compras " +
"using compras_view as cmp_view " +
"ON compras.id = cmp_view.id " +
"WHEN MATCHED THEN " +
"UPDATE SET compras.product = cmp_view.product," +
"compras.price = cmp_view.price " +
"WHEN NOT MATCHED THEN INSERT * ";
spark.sql(mergedados);

mergedados: String = "Merge into compras using compras_view as cmp_view ON compras.id = cmp_view.id WHEN MATCHED THEN UPDATE SET compras.product = cmp_view.product,compras.price = cmp_view.price WHEN NOT MATCHED THEN INSERT * "
res3: org.apache.spark.sql.DataFrame = [num_affected_rows: bigint, num_updated_rows: bigint ... 2 more fields]

## Exibe os dados que foram carregados com o merge

In [0]:
%scala
spark.sql("select * from compras").show();

+---+----------+--------+---------------+----+-----+
 id|date_order|customer| product|unit|price|
+---+----------+--------+---------------+----+-----+
 2|2021-03-22| Ana|AR-CONDICIONADO| 6|121.6|
 5|2021-05-23| Tereza| FRIGOBAR| 3|412.0|
 7|2021-07-25| Sofia| CADEIRA| 1|342.3|
 3|2021-04-21| Sofia| FREEZER| 7|415.4|
 6|2021-06-25| Carla| MESA| 1|124.0|
 4|2021-04-23| Sandra| TV| 8|313.0|
 1|2021-01-23| Carlos| TV| 5|238.0|
+---+----------+--------+---------------+----+-----+

##Atualiza os dados do id=4 com o comando update

In [0]:
%scala
val atualiza_dados = "update compras " +
"set product = 'Geladeira' " +
"where id =4";
spark.sql(atualiza_dados);

atualiza_dados: String = update compras set product = 'Geladeira' where id =4
res6: org.apache.spark.sql.DataFrame = [num_affected_rows: bigint]

## Exibe os dados que foram carregados, note a atualização no id=4

In [0]:
%scala
spark.sql("select * from compras").show();

+---+----------+--------+---------------+----+-----+
 id|date_order|customer| product|unit|price|
+---+----------+--------+---------------+----+-----+
 2|2021-03-22| Ana|AR-CONDICIONADO| 6|121.6|
 4|2021-04-23| Sandra| Geladeira| 8|313.0|
 5|2021-05-23| Tereza| FRIGOBAR| 3|412.0|
 7|2021-07-25| Sofia| CADEIRA| 1|342.3|
 3|2021-04-21| Sofia| FREEZER| 7|415.4|
 6|2021-06-25| Carla| MESA| 1|124.0|
 1|2021-01-23| Carlos| TV| 5|238.0|
+---+----------+--------+---------------+----+-----+

## Eliminação do registro cujo o id=4

In [0]:
%scala
val deletaregistro = "delete from compras where id = 1";
spark.sql(deletaregistro);

deletaregistro: String = delete from compras where id = 1
res9: org.apache.spark.sql.DataFrame = [num_affected_rows: bigint]

## Exibe os dados que foram carregados

In [0]:
%scala
spark.sql("select * from compras").show();

+---+----------+--------+---------------+----+-----+
 id|date_order|customer| product|unit|price|
+---+----------+--------+---------------+----+-----+
 2|2021-03-22| Ana|AR-CONDICIONADO| 6|121.6|
 4|2021-04-23| Sandra| Geladeira| 8|313.0|
 5|2021-05-23| Tereza| FRIGOBAR| 3|412.0|
 7|2021-07-25| Sofia| CADEIRA| 1|342.3|
 3|2021-04-21| Sofia| FREEZER| 7|415.4|
 6|2021-06-25| Carla| MESA| 1|124.0|
+---+----------+--------+---------------+----+-----+